In [11]:
import json
import os
import subprocess

from pprint import pprint

# Bit
#   https://github.com/ofek/bit
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI

# Web3
#   https://github.com/ethereum/web3.py
from web3 import Account, middleware, Web3
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.middleware import geth_poa_middleware

from dotenv import find_dotenv, get_key, load_dotenv
mnemonic = get_key(find_dotenv(), 'mnemonic')
mnemonic

'report excuse roast fruit tonight level next tomato raise habit cause across'

In [2]:
! php -version

PHP 7.4.20 (cli) (built: Jun  4 2021 03:32:07) ( NTS )
Copyright (c) The PHP Group
Zend Engine v3.4.0, Copyright (c) Zend Technologies
    with Zend OPcache v7.4.20, Copyright (c), by Zend Technologies


In [3]:
# hd-wallet-derive
#   https://github.com/dan-da/hd-wallet-derive

! hd-wallet-derive/hd-wallet-derive.php -g --key=xprv9tyUQV64JT5qs3RSTJkXCWKMyUgoQp7F3hA1xzG6ZGu6u6Q9VMNjGr67Lctvy5P8oyaYAL9CAWrUE9i6GoNMKUga5biW6Hx4tws2six3b9c --numderive=3 --preset=bitcoincore --cols=path,address --path-change


+------------+------------------------------------+
| path       | address                            |
+------------+------------------------------------+
| m/0'/1'/0' | 1B6q1KTyaa9yLHV2HTZC1rZaSKMG8KNqsp |
| m/0'/1'/1' | 15RF1R9ZaSqgtaTVBDm1ySU5MQ6dZeTpZf |
| m/0'/1'/2' | 1DpzhgrgWuRSnQjvLiZHMG2TAjs86znvjj |
+------------+------------------------------------+



In [4]:
BTC = 'btc'
BTCTEST = 'btc-test'
ETH = 'eth'

In [5]:
w3 = Web3(Web3.HTTPProvider('http://localhost:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [6]:
def derive_wallets (coin=BTC, mnemonic=mnemonic, depth=3):
    command = f'php hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depth} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    return json.loads(output)

def priv_key_to_account (coin, priv_key):
    """Convert priv_key to an account object that bit or web3 can use to transact.
    Parameters
    ==========
    coin [str]: the coin type
    priv_key [str]: a child key's private key
    """
    if coin == ETH:
        # https://web3js.readthedocs.io/en/v1.2.0/web3-eth-accounts.html#privatekeytoaccount
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        # convert the private key into a WIF Wallet Import Format object
        # ...which is a format used to designate the type of key
        # https://ofek.dev/bit/dev/api.html
        return PrivateKeyTestnet(priv_key)
    
def create_tx (coin, account, to, amount):
    """Create a raw, unsigned transaction that contains the necessary metadata.
    Parameters
    ==========
    coin [str]: the coin type
    account [account object]: from priv_key_to_account
    to [address]: the recipient address
    amount: the amount of the coin to send
    """
    if coin == ETH:
        value = w3.toWei(amount, 'ether')
        gasEstimate = w3.eth.estimateGas({'to': to, 'from': account, 'amount': value})
        return {
            'to': to,
            'from': account,
            'value': value,
            'gas': gasEstimate,
            'gasPrice': w3.eth.generateGasPrice(),
            'nonce': w3.eth.getTransactionCount(account),
            'chainId': w3.eth.chain_id,
        }
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])
    
def send_tx (coin, account, to, amount):
    """Call create_tx, sign the transaction, and send it to the designated network.
    Parameters
    ==========
    coin [str]: the coin type
    account [account object]: from priv_key_to_account
    to [address]: the recipient address
    amount: the amount of the coin to send
    """
    if coin == ETH:
        raw_tx = create_tx(coin, account.address, to, amount)
        signed = account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)

In [7]:
coins = {
    ETH: derive_wallets(coin=ETH),
    BTCTEST: derive_wallets(coin=BTCTEST),
}

In [8]:
pprint(coins)

{'btc-test': [{'address': 'mn7fdQhacxTGqcCapzEqw91bh6Fx3saUCm',
               'index': 0,
               'path': "m/44'/1'/0'/0/0",
               'privkey': 'cSu7cNuekRTPXM3g9NgHrruaMQEsa8U3J5hYDUgAsXJTtA2ibE69',
               'pubkey': '03b681c2cad2961fc814ce3936ac62a5d1a3fd1d4fd774e93186bbd2d89093696c',
               'pubkeyhash': '486166ab3c0ae6843435a9d4d163207d005faffa',
               'xprv': 'tprv8jHfJwVdPi5TSenL7ghXs3qg9Ap8myN4t4sYLijjmhGHSrJZfRKJuTXVVsDukfBJV1iSmD9QVJYNkNQ4oyzN7xSB3GDQ6BwQmj8bJvokDpG',
               'xpub': 'tpubDFyhTMXsY5m8L7p81LN8GTVniCL4wJYyTNUKdEn3By4gHLZLHp8u5x9Mg2J43Tzkv1Hkknif4z9cnaAzV9Qy7fNoGK77fQ7Vw9NbikgYv7A'},
              {'address': 'mxV6Shpp6dLsCGX811ofEsdCGJgsBJwQHi',
               'index': 1,
               'path': "m/44'/1'/0'/0/1",
               'privkey': 'cPtVkGvH238hSFcTJ2qesArWseqWSVLSohJEFWYfrFYY1NBNCJGy',
               'pubkey': '025fb7f21e7c2d764628c7934a6353d8db93bd88677ba74eb484a4cf44a0ce8c58',
               'pubkeyhash': '

In [9]:
account = priv_key_to_account(BTCTEST, coins[BTCTEST][0]['privkey'])
account.__dict__

{'_pk': <coincurve.keys.PrivateKey at 0x7f98541d6700>,
 '_public_point': None,
 '_public_key': b'\x03\xb6\x81\xc2\xca\xd2\x96\x1f\xc8\x14\xce96\xacb\xa5\xd1\xa3\xfd\x1dO\xd7t\xe91\x86\xbb\xd2\xd8\x90\x93il',
 'version': 'test',
 'instance': 'PrivateKeyTestnet',
 '_address': None,
 '_segwit_address': None,
 '_scriptcode': None,
 '_segwit_scriptcode': None,
 'balance': 0,
 'unspents': [],
 'transactions': []}

In [10]:
send_tx(BTCTEST, account, 'mv4rnyY3Su5gjcDNzbMLKBQkBicCtHUtFB', '0.001')

ValueError: Transactions must have at least one unspent.